# Task Details

The goal here is to extract business insights from the WorldWide Importers database, and present them in a format that non-analysts can understand. In order to do this, we'll be doing the following:

1. Deciding what information we need from the dataset
2. Extracting the data with SQL queries
3. Exporting the SQL queries as a CSV file
4. Graphing the data with a BI software (Tableau, Power BI)

Sales Questions:
- What are our totals?
    - Sales?
    - Revenue?
    - Costs?
- What are our top selling categories?
    - Overall?
    - Compared to previous years?
- Top customers
    - By year
    
I'll be documenting my SQL queries here.


In [2]:
## Import os and pandas
import os
import pandas as pd, pyodbc

In [3]:
## create function to read in SQL to python
def get_sql(sql_path, print_sql=True):
    assert os.path.exists(sql_path), f'{sql_path} does not exists'
    _in = open(sql_path,'r')
    sql = _in.read()
    _in.close()
    if print_sql:
        print(sql)
    return sql

In [4]:
#create function to establish connection to database Worldwide Importers
def get_wwi_cnn():
    ''' get connection to world wide importers'''
    cnn_str = ("Driver={SQL Server Native Client 11.0};"
                "Server=DESKTOP-6IFNJ56\SQLEXPRESS;"
                "Database=WideWorldImporters;"
                "Trusted_Connection=yes;")
    cnn = pyodbc.connect(cnn_str)
    return cnn

#assign to cnn_wwi
cnn_wwi = get_wwi_cnn()

now, we can create queries and save to SQL here

## Question 1

What are our total sales?

In [5]:
sql= """select year(lasteditedwhen) 'year', sum(extendedprice) 'revenue'
from sales.InvoiceLines
group by year(lasteditedwhen)
order by 2 desc"""

In [6]:
#read query into sql
df = pd.read_sql(sql, cnn_wwi)
#save file
df.to_csv('total_sales.csv')

## Question 1b

What are our total profits?

In [7]:
sql= """
select year(lasteditedwhen) 'year', sum(LineProfit) 'profit'
from sales.InvoiceLines
group by year(lasteditedwhen)
order by 2 desc
"""

In [8]:
#read query into sql
df = pd.read_sql(sql, cnn_wwi)
#save file
df.to_csv('total_profits.csv')

## Question 1c

What are our sales compared to previous years, between 2013 and 2015?

In [17]:
sql="""
select year(lasteditedwhen) 'year', sum(extendedprice) 'current_year_sales', lag(sum(extendedprice),1) over (order by year(lasteditedwhen)) 'prev_year_sales'
from sales.InvoiceLines
where year(lasteditedwhen) between '2013' and '2015'
group by year(lasteditedwhen)
"""

In [18]:
#read query into sql
df = pd.read_sql(sql, cnn_wwi)
#save file
df.to_csv('prev_year_sales.csv')

## Question 2

What are our top selling groups?

In [9]:
sql = """
select grp.stockgroupname, sum(il.extendedprice) total_amount
from sales.InvoiceLines il
join warehouse.StockItems si on il.StockItemID = si.StockItemID
join warehouse.StockItemStockGroups sg on si.StockItemID = sg.StockItemID
join warehouse.StockGroups grp on sg.StockGroupID = grp.StockGroupID
group by grp.StockGroupName
order by 2 desc
"""

In [10]:
#read query into sql
df = pd.read_sql(sql, cnn_wwi)
#save file
df.to_csv('top_selling_groups.csv')

## Question 2a

What are our selling groups for each year?

In [11]:
sql = """
select year(il.LastEditedwhen), grp.stockgroupname, sum(il.ExtendedPrice) total_amount
from sales.InvoiceLines il
join warehouse.StockItems si on il.StockItemID = si.StockItemID
join warehouse.StockItemStockGroups sg on si.StockItemID = sg.StockItemID
join warehouse.StockGroups grp on sg.StockGroupID = grp.StockGroupID
where year(il.LastEditedwhen) is not null
group by year(il.LastEditedwhen), grp.StockGroupName
order by 2 desc
"""

In [12]:
#read query into sql
df = pd.read_sql(sql, cnn_wwi)
#save file
df.to_csv('top_selling_groups_year.csv')

## Question 3

Who are our top 50 customers by revenue?


In [13]:
sql = """
select top 50 c.customername, sum(il.extendedprice) 'revenue'
from sales.InvoiceLines il
join sales.Invoices i on il.InvoiceID = i.InvoiceID
join sales.customers c on c.customerid = i.CustomerID
group by c.CustomerName
order by 2 desc"""

In [14]:
#read query into sql
df = pd.read_sql(sql, cnn_wwi)
#save file
df.to_csv('top_50_customers.csv')

## Combined Table

We'll combine all the individual tables for processing into Tableau for analysis.

Tables needed:
- sales.invoices
- sales.customers
- sales.customercategories
- warehouse.stockitems
- warehouse.stockitemstockgroups
- warehouse.stockgroups

## Combined Table Query

In [23]:
sql = """select
--invoiceline
il.invoicelineid, il.invoiceid, il.stockitemid, il.extendedprice, il.StockItemID, il.LastEditedWhen, il.LineProfit
--invoices
,i.invoiceid, i.customerid, i.orderid
--customer
,c.customerid, c.customername, c.deliverypostalcode, c.customercategoryid
--customercategories
,cc.customercategoryid, cc.customercategoryname
--stockitems
,si.stockitemid,si.stockitemname
--stockitemstockgroups
,sisg.stockitemid, sisg.stockgroupid
--stockgroups
,sg.stockgroupid,sg.stockgroupname

from sales.InvoiceLines il
left join sales.invoices i on il.invoiceid = i.invoiceid
left join sales.Customers c on i.customerid = c.customerid
left join sales.customercategories cc on c.CustomerCategoryID = cc.customercategoryid
left join warehouse.stockitems si on si.StockItemID = il.StockItemID
left join warehouse.StockItemStockGroups sisg on si.StockItemID = sisg.StockItemID
left join warehouse.StockGroups sg on  sisg.StockGroupID = sg.StockGroupID
"""

df = pd.read_sql(sql, cnn_wwi)

In [22]:
df.to_csv('wwi_data.csv')

## Dashboard

I have generated a quick Tableau dashboard with the SQL query above. You can access it from my github portfolio!